Utilisation des dernières versions de librairies, et inspiration de :
- https://python.langchain.com/docs/how_to/output_parser_json/


In [1]:
from langchain.chat_models import init_chat_model
from typing import Optional
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "voc-8162499801266773377505669655d3c05508.40840521"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [3]:
model_name = "gpt-3.5-turbo-instruct" # "gpt-4o-mini"
temperature = 0.0
llm = init_chat_model(model_name, model_provider="openai", temperature=temperature, max_tokens=3000)

In [4]:
class PropertyAdvertClass(BaseModel):
    """Real estate property advertisement class."""
    neighborhood: str = Field(
        description = "location in USA including the name the neighborhood"
    )
    style: str = Field(
        description = "style of construction"
    )
    rooms: int = Field(
        description = "number of rooms"
    )
    bedrooms: int = Field(
        description = "number of bedrooms"
    )
    bathrooms: int = Field(
        description = "number of bathrooms"
    )
    floors: int = Field(
        description = "number of floors"
    )
    house_size: str = Field(
        description = "surface area in square feet"
    )
    price: str = Field(
        description = "price in dollars"
    )
    property_description : str = Field(
        description = "a detailed description of the property"
    )
    neighborhood_description : str = Field(
        description = "the neighborhood description"
    )

In [5]:
structured_llm = llm.with_structured_output(PropertyAdvertClass)
parser = JsonOutputParser(pydantic_object=PropertyAdvertClass)

/Users/me/opt/miniconda3/envs/udacity_genai_p5/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1645: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-instruct since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [ ]:
prompt = PromptTemplate(
    template="{question}\n{format_instructions}",
    input_variables=["question", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)

num_ads = 3
adverts_query = f"""
    generate {num_ads} property advertisements for middle-class buyers, each respecting the output schema, gathered in a json array.
    be creative in your descriptions but consistent and realistic.
"""
chain = prompt | llm | parser

adverts = chain.invoke({"query": adverts_query, "context": "The following is a list of properties for sale in the USA."})
print(adverts)